In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter

In [2]:
import sys
sys.argv = [""]

In [3]:
from cart_pole.utils import initialize
from numpy.random import Generator, MT19937, SeedSequence
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import make_sync_vector_env

In [8]:
args = initialize()

In [9]:
seed_seq = SeedSequence(args.seed)
rng = Generator(MT19937(seed_seq))

In [11]:
envs = make_sync_vector_env(args)

In [12]:
writer = SummaryWriter(f"runs/{run_name}")
writer.add_text(
    "hyperparameters",
    "|param|value|\n|-|-|\n%s" % ("\n".join([f"|{key}|{value}|"
                                             for key, value in
                                             vars(args).items()])),
)

In [16]:
seeds = [int(rng.uniform(0, 1E9)) for _ in np.arange(args.num_envs)]
envs.reset(seed=seeds)

(array([[-0.02383922, -0.02171686,  0.01937895, -0.04366146],
        [-0.02111782, -0.01734802,  0.03308019, -0.02281427],
        [-0.0091763 ,  0.01347467,  0.01009424,  0.00192122],
        [ 0.02736252,  0.0082393 ,  0.02533039, -0.04266285]],
       dtype=float32),
 {})